In [1]:
import re, fnmatch
import pandas as pd
import requests
from statistics import mean
import csv
import numpy as np
import glob
import os
import scipy.stats as stats
from scipy.stats import mannwhitneyu
import math
import json

In [3]:
fake = pd.read_csv('/workspaces/fake_news_analysis/data/preprocessed/fake_preprocessed.csv')

KeyboardInterrupt: 

In [ ]:
real = pd.read_csv('/workspaces/fake_news_analysis/data/preprocessed/real_preprocessed.csv')

In [ ]:
# dictionaries
dictionary_paths = {
    'mft': '/workspaces/fake_news_analysis/dictionaries/processed/mft_dictionary.json',
    'mfd2': '/workspaces/fake_news_analysis/dictionaries/processed/mfd2_dictionary.json',
    'mfd1': '/workspaces/fake_news_analysis/dictionaries/processed/mfd1_dictionary.json',
    'emfd': '/workspaces/fake_news_analysis/dictionaries/processed/emfd_dictionary.json',
    'ms': '/workspaces/fake_news_analysis/dictionaries/processed/ms_dictionary.json'
}


loaded_dictionaries = {}
for key, path in dictionary_paths.items():
    with open(path, 'r') as file:
        loaded_dictionaries[key] = json.load(file)

In [ ]:
def score_vector(tokens, dictionary):
    scores = {dimension: {polarity: 0 for polarity in dictionary[dimension]} for dimension in dictionary}

    for token in tokens:
        for dimension, polarities in dictionary.items():
            for polarity, words in polarities.items():
                if isinstance(words, list) and words:
                    for word in words:
                        if isinstance(word, tuple):
                            word = word[0]  # Extract the word from the tuple if it is a tuple
                        if token == word:
                            scores[dimension][polarity] += 1

    vector = []
    for dimension in scores.keys():
        for polarity in scores[dimension].keys():
            vector.append(scores[dimension][polarity])
    
    return vector

In [ ]:
def tracked_words(tokens, dictionary):
    tracked = {dimension: {polarity: [] for polarity in dictionary[dimension]} for dimension in dictionary}

    for token in tokens:
        for dimension, polarities in dictionary.items():
            for polarity, words in polarities.items():
                if isinstance(words, list) and words:
                    for word in words:
                        if isinstance(word, tuple):
                            word = word[0]
                        if token == word and token not in tracked[dimension][polarity]:
                            tracked[dimension][polarity].append(token)
    
    return tracked


In [ ]:
# Assuming df['processed_text'] contains the list of tokens for each article
df['mft_vectors'] = df['processed_text'].apply(lambda x: score_vector(x, loaded_dictionaries['mft']))
df['mft_tracked_words'] = df['processed_text'].apply(lambda x: tracked_words(x, loaded_dictionaries['mft']))